# Keras中的遮盖和填充

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

__遮盖__：告知序列处理层输入中有些时间步骤丢失，因此在处理数据时应将其跳过

__填充__：遮盖的特殊形式，其中被遮盖的步骤位于序列的起点或开头。填充是出于将序列数据编码成连续批次的需要：为了使批次中的所有序列适合给定的标准长度，有必要填充或截断某些序列。

## 填充序列数据

例如数据是嵌套列表，各个样本的长度不一。但深度学习模型的输入数据必须是单一张量，短于最长条目的样本需要用占位符值进行填充，或者在填充短样本前截断长样本。

In [2]:
raw_inputs = [
    [711, 632, 71],
    [73, 9, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927]
]

padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs, padding="post")
print(padded_inputs)

[[ 711  632   71    0    0    0]
 [  73    9 3215   55  927    0]
 [  83   91    1  645 1253  927]]


## 遮盖

经过填充后，需要告知模型，哪些部分实际上是填充的，应该忽略。这种机制就是遮盖。

Keras中遮盖的三种方式：

* 添加一个`keras.layers.Masking`层
* 使用`mask_zero=True`配置一个`keras.layers.Embedding`层
* 在调用支持`mask`参数的层时，手动传递参数

## 掩码生成层：Embedding和Masking

在后台创建一个掩码张量（`(batch, sequence_length)`的二维张量），将其附加到由`Masking`或`Embedding`层返回的张量输出上。

In [3]:
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)

print(masked_output._keras_mask)

masking_layer = layers.Masking()

unmasked_embedding = tf.cast(tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1,1,10]), tf.float32)

masked_embedding = masking_layer(unmasked_embedding)
print(masked_embedding._keras_mask)

tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)
tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)


## 函数式API和序列式API中的掩码传播

In [4]:
# 序列模型
model = keras.Sequential(
    [layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True), layers.LSTM(32)]
)

In [5]:
# 函数式API
inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
outputs = layers.LSTM(32)(x)

model = keras.Model(inputs, outputs)

## 将掩码张量直接传递给层

能够处理掩码的层，如`LSTM`层，在其`__call__`方法中有一个`mask`参数。

在生成掩码的层，如`Embedding`，会公开一个`compute_mask(input, previous_mask)`方法。

可以将掩码生成层的`compute_mask()`方法的输出传递给掩码使用层的`__call__`方法。

In [6]:
class MyLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        self.embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
        self.lstm = layers.LSTM(32)

    def call(self, inputs):
        x = self.embedding(inputs)
        # mask tensor可以手动准备，但是请注意其形状
        mask = self.embedding.compute_mask(inputs)
        output = self.lstm(x, mask=mask)
        return output

layer = MyLayer()
x = np.random.random((32, 10)) * 100
x = x.astype("int32")
layer(x)

<tf.Tensor: shape=(32, 32), dtype=float32, numpy=
array([[ 0.00654859,  0.0051147 ,  0.00080724, ..., -0.00455279,
         0.0029917 ,  0.0041261 ],
       [ 0.01040531, -0.00743341,  0.01023701, ...,  0.00405205,
        -0.00841662, -0.00174078],
       [ 0.00652924,  0.00207402,  0.00406809, ..., -0.0088197 ,
         0.00276266, -0.01376904],
       ...,
       [-0.01551128,  0.00162226,  0.00389638, ...,  0.00463707,
        -0.00219373, -0.0003141 ],
       [ 0.00514781,  0.00777179,  0.00195215, ..., -0.00056386,
        -0.00011741, -0.0039389 ],
       [ 0.00282336,  0.00047061,  0.00593017, ..., -0.00324067,
         0.00156855,  0.00412254]], dtype=float32)>

## 在自定义层中支持遮盖

任何生成与其具有不同时间维度的张量的层，都需要修改当前的掩码，这样下游层才能正确估计被遮盖的时间步骤。

自定义层需要实现`layer.compute_mask()`方法，还方法依据输入和当前掩码生成新的掩码。

In [8]:
class TemporalSplit(layers.Layer):
    # 沿时间维度，将输入张量划分成两个张量
    def call(self, inputs):
        return tf.split(inputs, 2, axis=1)

    def compute_mask(self, inputs, mask=None):
        if mask is None:
            return None
        return tf.split(mask, 2, axis=1)

first_half, second_half = TemporalSplit()(masked_embedding)
print(first_half._keras_mask)
print(second_half._keras_mask)

tf.Tensor(
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]], shape=(3, 3), dtype=bool)
tf.Tensor(
[[False False False]
 [ True  True False]
 [ True  True  True]], shape=(3, 3), dtype=bool)


In [ ]:
class CustomEmbedding(layers.Layer):
    def __init__(self, input_dim, output_dim, mask_zero=False, **kwargs):
        super(CustomEmbedding, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.mask_zero = mask_zero

    def build(self, input_shape):
        self.embeddings = self.add_weight(
            shape=(self.input_dim, self.output_dim),
            initializer="random_normal",
            dtype="float32"
        )

    def call(self, inputs):
        return tf.nn.embedding_lookup(self.embeddings, inputs)

    def compute_mask(self, )